In [146]:
import pandas as pd

In [147]:
suffixes_to_remove = ["?fbclid=", "+&", "?_x_tr_", "?back="]
search_cache_identifier = "/search?q=cache:"
sacommunity_url = "https://sacommunity.org"


In [148]:
def clean_landing_page_column(text: str) -> str:
    if search_cache_identifier in text:
        text = text[text.index(sacommunity_url):].replace(sacommunity_url, "")

    for suffix_to_remove in suffixes_to_remove:
        if suffix_to_remove in text:
            text = text[:text.index(suffix_to_remove)]

    # remove underscore
    text = text.replace("_", " ")
    # remove /org/
    text = text.replace("/org/", "")

    return text

def get_organization_id(text: str) -> str:
    if "-" in text:
        return int(text[:text.index("-")])
    else:
        return None
    
def get_organization_name(text: str) -> str:
    if "-" in text:
        return text[text.index("-") + 1:]
    else:
        return None

# test texts
inputs = [
    "/org/196236-Dave's_Angels_Playgroup?fbclid=IwAR05WAQ0z5mwY7v1UEVmkDITFg7sDh8pcD8taJ3oGH4336EpkNZeP81BIKc",
    "/search?q=cache:UTs_a-1ZNgEJ:https://sacommunity.org/org/196341-Neighbourhood_Watch_-_Linden_Park_249+&cd=63&hl=en&ct=clnk&gl=bj",
    "/org/201669-Gifted_&_Talented_Children's_Association_of_SA_Inc.?_x_tr_sl=en&_x_tr_tl=th&_x_tr_hl=th&_x_tr_pto=sc",
    "/org/201830-Aged_Rights_Advocacy_Service_Inc.?back=https://www.google.com/search?client=safari&as_qdr=all&as_occt=any&safe=active&as_q=Age+advocate+for+South+Australia&channel=aplab&source=a-app1&hl=en",
    "/org/201950-SA_Ambulance_Service?_x_tr_sl=en&_x_tr_tl=fr&_x_tr_hl=fr&_x_tr_pto=nui,sc"
]
for input in inputs:
    print(clean_landing_page_column(input))
    

196236-Dave's Angels Playgroup
196341-Neighbourhood Watch - Linden Park 249
201669-Gifted & Talented Children's Association of SA Inc.
201830-Aged Rights Advocacy Service Inc.
201950-SA Ambulance Service


In [149]:
landing_page_file_path = './data/Burnside Council Landing page 2021-2022.xlsx'
df_google_analytics = pd.read_excel(landing_page_file_path, sheet_name='Dataset1')
df_google_analytics

/Users/pujanmaharjan/anaconda/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Custom Variable (Value 01),Landing Page,Sessions,% New Sessions,New Users,Bounce Rate,Pages / Session,Avg. Session Duration,Goal Conversion Rate,Goal Completions,Goal Value
0,0QK91R12,/org/201829-Cats_Assistance_To_Sterilise_Inc.,632,0.829114,524,0.794304,1.229430,49.006329,0,0,0
1,0QK91R12,/org/202703-Drug_and_Alcohol_Services_SA_-_Wit...,295,0.762712,225,0.755932,1.223729,82.186441,0,0,0
2,0QK91R12,/org/236722-Justices_of_the_Peace_-_City_of_Bu...,237,0.898734,213,0.869198,1.071730,35.308017,0,0,0
3,0QK91R12,/org/201950-SA_Ambulance_Service,234,0.871795,204,0.705128,1.376068,64.128205,0,0,0
4,0QK91R12,/org/201612-Dementia_Australia,214,0.827103,177,0.836449,1.140187,38.948598,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
234,0QK91R12,/org/234010-MatchWorks_-_Tusmore,1,0.000000,0,1.000000,1.000000,0.000000,0,0,0
235,0QK91R12,/org/235748-Rostrum_Club_32_-_Tusmore,1,0.000000,0,1.000000,1.000000,0.000000,0,0,0
236,0QK91R12,/org/236332-Gentle_Touch_Orthodontics,1,1.000000,1,1.000000,1.000000,0.000000,0,0,0
237,0QK91R12,/search?q=cache:UTs_a-1ZNgEJ:https://sacommuni...,1,1.000000,1,1.000000,1.000000,0.000000,0,0,0


In [150]:

def clean_google_analytics_file(df_ga_orig: pd.DataFrame) -> pd.DataFrame:
    df_ga = df_ga_orig.dropna().copy()
    df_ga['organization_id_name'] = df_ga['Landing Page'].apply(clean_landing_page_column)
    df_ga['organization_id'] = df_ga['organization_id_name'].apply(get_organization_id)
    df_ga['organization_name'] = df_ga['organization_id_name'].apply(get_organization_name)
    return df_ga[["Landing Page", "organization_id_name","organization_id","organization_name", "Sessions"]]

In [151]:
df_cleaned = clean_google_analytics_file(df_google_analytics)
df_cleaned

,Landing Page,organization_id_name,organization_id,organization_name,Sessions
0,/org/201829-Cats_Assistance_To_Sterilise_Inc.,201829-Cats Assistance To Sterilise Inc.,201829,Cats Assistance To Sterilise Inc.,632
1,/org/202703-Drug_and_Alcohol_Services_SA_-_Wit...,202703-Drug and Alcohol Services SA - Withdraw...,202703,Drug and Alcohol Services SA - Withdrawal Serv...,295
2,/org/236722-Justices_of_the_Peace_-_City_of_Bu...,236722-Justices of the Peace - City of Burnside,236722,Justices of the Peace - City of Burnside,237
3,/org/201950-SA_Ambulance_Service,201950-SA Ambulance Service,201950,SA Ambulance Service,234
4,/org/201612-Dementia_Australia,201612-Dementia Australia,201612,Dementia Australia,214
...,...,...,...,...,...
233,/org/231232-SA_Metropolitan_Fire_Service_-_Beu...,231232-SA Metropolitan Fire Service - Beulah Park,231232,SA Metropolitan Fire Service - Beulah Park,1
234,/org/234010-MatchWorks_-_Tusmore,234010-MatchWorks - Tusmore,234010,MatchWorks - Tusmore,1
235,/org/235748-Rostrum_Club_32_-_Tusmore,235748-Rostrum Club 32 - Tusmore,235748,Rostrum Club 32 - Tusmore,1
236,/org/236332-Gentle_Touch_Orthodontics,236332-Gentle Touch Orthodontics,236332,Gentle Touch Orthodontics,1


In [152]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 0 to 237
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Landing Page          238 non-null    object
 1   organization_id_name  238 non-null    object
 2   organization_id       238 non-null    int64 
 3   organization_name     238 non-null    object
 4   Sessions              238 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ KB


In [153]:
df_cleaned[df_cleaned['organization_id'] == 196236]

,Landing Page,organization_id_name,organization_id,organization_name,Sessions
40,/org/196236-Dave's_Angels_Playgroup,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,41
201,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
202,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
203,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
204,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
205,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
206,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
207,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
208,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1
209,/org/196236-Dave's_Angels_Playgroup?fbclid=IwA...,196236-Dave's Angels Playgroup,196236,Dave's Angels Playgroup,1


In [168]:
def grp_by_org_id_and_sum_sessions(df_cleaned: pd.DataFrame) -> pd.DataFrame:
    df_grp_org_id = df_cleaned.groupby(by=['organization_id']).sum("Sessions")
    # df_grp_org_id = df_grp_org_id.reset_index()
    return df_grp_org_id

In [169]:
df_grp_by_org_id = grp_by_org_id_and_sum_sessions(df_cleaned)
df_grp_by_org_id

,Sessions
organization_id,
194813,34
196167,11
196171,30
196173,4
196174,4
...,...
236280,58
236332,1
236722,237


In [173]:
df_grp_by_org_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 194813 to 237449
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Sessions  205 non-null    int64
dtypes: int64(1)
memory usage: 3.2 KB


In [182]:
df_grp_by_org_id[df_grp_by_org_id.index == 194813]

,Sessions
organization_id,
194813,34


In [192]:
df_grp_by_org_id[df_grp_by_org_id.index == 1948132].values

array([], shape=(0, 1), dtype=int64)

In [196]:
len(df_grp_by_org_id[df_grp_by_org_id.index == 194813].values)

1

In [195]:
df_grp_by_org_id[df_grp_by_org_id.index == 194813].values[0][0]

34

In [167]:
df_grp_by_org_id.sort_values(by=["organization_id"]).head(80)

,organization_id,Sessions
0,194813,34
1,196167,11
2,196171,30
3,196173,4
4,196174,4
...,...,...
75,196326,5
76,196337,11
77,196340,4
78,196341,1


In [ ]:
# for all the duplicates get data from sa community export
# easy solution, get all organization name from sacommunity exports



In [198]:
df_sacommunity_data = pd.read_csv('./data/Burnside Council-cu_export_2023-08-16_Data.Gov.au_export.csv')
df_sacommunity_data

,ID_19,Org_name,Street_Address_Line_1,Street_Address_Line_2,Suburb,State,Postal_Code,Country,Postal_Address_Line_1,Postal_Address_Line_2,...,Organisati_Eligibility,Organisati_Services,Organisation_Created_Date,Organisation_Last_updated,IM_Screen_Name_1,IM_Screen_Name_2,IM_Screen_Name_3,IM_Screen_Name_4,IM_Screen_Name_5,IM_Screen_Name_6
0,194023,Motorcycling South Australia Inc.,"Motorcycling South Australia Inc., 251 The Pde",NaN,Beulah Park,South Australia,5067.0,Australia,NaN,NaN,...,NaN,Coordination and promotion of motorcycle sport...,2009-11-23 21:44:28,2022-11-16 11:54:08,https://www.facebook.com/motorcyclingsa/ 7,NaN,NaN,NaN,NaN,NaN
1,194025,Orienteering SA,NaN,NaN,Glenside,South Australia,5065.0,Australia,c/o 5/355 Angas St,NaN,...,NaN,Coordination and promotion of orienteering in ...,2009-11-23 21:44:28,2022-07-24 15:14:31,https://www.facebook.com/OrienteeringSA 7,https://www.youtube.com/channel/UCSrZVpB1et3Jo...,NaN,NaN,NaN,NaN
2,194813,Burnside Library & Information Service,401 Greenhill Rd,NaN,Tusmore,South Australia,5065.0,Australia,PO Box 9,NaN,...,NaN,Public library\r\nHome Energy Toolkit availabl...,2009-11-23 21:44:58,2022-12-07 10:38:10,BurnsideLibrary 7,BurnsideLibrary 4,burnsidelibrary 1,NaN,NaN,NaN
3,196167,Burnside Family Church,88 Lockwood Rd,NaN,Burnside,South Australia,5066.0,Australia,NaN,NaN,...,NaN,Sunday Service 10am\r\nFri night kid's club an...,2009-11-23 21:45:53,2023-01-25 13:53:04,burnsidefamilychurch 7,BFamilyChurch 4,NaN,NaN,NaN,NaN
4,196171,Linden Park Primary School and OSHC/VAC,"Linden Park Primary School, 14 Hay Rd",NaN,Linden Park,South Australia,5065.0,Australia,NaN,NaN,...,NaN,Primary education - Reception to Year 7\r\nOut...,2009-11-23 21:45:53,2023-01-25 14:06:43,https://www.facebook.com/groups/3219860600/abo...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,237283,Salvos Stores - Kensington Gardens,418 Magill Rd,NaN,Kensington Gardens,South Australia,5068.0,Australia,NaN,NaN,...,NaN,"Clothing\r\nBric-a-Brac & Homewares\r\nBooks, ...",2021-08-11 13:53:47,2021-08-24 13:57:38,https://www.facebook.com/TheSalvationArmySouth...,NaN,NaN,NaN,NaN,NaN
210,237449,Military and Emergency Services Health Australia,Jamie Larcombe Centre,Eucalyptus Ln,Glenside,South Australia,5056.0,Australia,THRF Group Head Office,PO Box 77,...,Current and former serving Australian military...,Research Institute\r\nOffers research-based pr...,2021-10-25 12:15:00,2022-11-29 22:42:28,NaN,NaN,NaN,NaN,NaN,NaN
211,238110,South Australian Chinese Community Language Sc...,NaN,384 Portrush Rd,Tusmore,South Australia,5065.0,Australia,384 Portrush Rd,NaN,...,NaN,Chinese language learning and cultural classes,2023-02-02 20:10:20,2023-04-03 11:41:22,NaN,NaN,NaN,NaN,NaN,NaN
212,238198,Resthaven Leabrook Retirement Living Apartments,334 Kensington Road,NaN,Kensington Park,South Australia,5068.0,Australia,NaN,NaN,...,Eligibility for retirement unit or apartment r...,8 retirement living apartments \r\ntwo or thr...,2023-03-30 13:37:02,2023-03-31 17:06:40,resthaven.sa 7,Resthaven_SA 4,resthaveninc 12,resthaven-incorporated 11,ResthavenInc 1,NaN


In [ ]:
df_sacommunity_data[df_sacommunity_data['ID_19'] == 196294]

In [101]:
df_sacommunity_data.columns

Index(['ID_19', 'Org_name', 'Street_Address_Line_1', 'Street_Address_Line_2',
       'Suburb', 'State', 'Postal_Code', 'Country', 'Postal_Address_Line_1',
       'Postal_Address_Line_2', 'Suburb.1', 'State.1', 'Postal_Code.1',
       'Country.1', 'Organisati_Hours', 'Organisati_Meetings',
       'Organisati_AGM', 'Geo_Code_1_StreetAddress_',
       'Geo_Code_2_StreetAddress_', 'Organisati_Venue_hire',
       'Organisati_Access', 'Organisati_Disability_parking',
       'Organisati_Access_toilets', 'Organisati_Wheelchair_access',
       'Organisati_Fees', 'Organisati_ABN', 'Contact', 'Phone_1',
       'Phone_1_comment', 'Phone_2', 'Phone_2_comment', 'Phone_3',
       'Phone_3_comment', 'Email_1', 'Email_2', 'Email_3', 'Email_4',
       'Website_1', 'Website_2', 'Website_3', 'Primary_Category',
       'Directory_Directory_Entry_1', 'Directory_Directory_Entry_2',
       'Directory_Directory_Entry_3', 'Parent_Body', 'Organisati_Tag_Line',
       'Organisati_Eligibility', 'Organisati_Service

In [102]:
df_sacommunity_data[['ID_19', 'Org_name', 'Primary_Category']]

,ID_19,Org_name,Primary_Category
0,194023,Motorcycling South Australia Inc.,Recreation
1,194025,Orienteering SA,Recreation
2,194813,Burnside Library & Information Service,Communication & Information Services
3,196167,Burnside Family Church,Religions & Philosophies
4,196171,Linden Park Primary School and OSHC/VAC,Education
...,...,...,...
209,237283,Salvos Stores - Kensington Gardens,Material & Practical Needs
210,237449,Military and Emergency Services Health Australia,Health & Disability
211,238110,South Australian Chinese Community Language Sc...,Education
212,238198,Resthaven Leabrook Retirement Living Apartments,Accommodation
